In [1]:
import darling
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.transform import Rotation
from darkmod import laue, reconstruct, scan
from darkmod.beam import GaussianLineBeam, HeavysideBeam
from darkmod.crl import CompundRefractiveLens
from darkmod.crystal import Crystal
from darkmod.deformation import straight_edge_dislocation
from darkmod.detector import Detector
from darkmod.resolution import PentaGauss
from darkmod.utils import crop
import matplotlib.pyplot as plt
plt.style.use("dark_background")
fontsize = 16  # General font size for all text
ticksize = 16  # tick size
plt.rcParams["font.size"] = fontsize
plt.rcParams["xtick.labelsize"] = ticksize
plt.rcParams["ytick.labelsize"] = ticksize

In [2]:
number_of_lenses = 69
lens_space = 1600  # microns
lens_radius = 50  # microns
magnification = 15.1

energy = 19.1  # keV
lambda_0 = laue.keV_to_angstrom(energy)

Z = 4  # atomic number, berillium
rho = 1.845  # density, berillium, g/cm^3
A = 9.0121831  # atomic mass number, berillium, g/mol
delta = laue.refractive_decrement(Z, rho, A, energy)

crl = CompundRefractiveLens(
    number_of_lenses, lens_space, lens_radius, delta, magnification
)



In [5]:
crl.d2

6508990.118235542

In [7]:

hkl = np.array([-1, -1, 3])
omega = np.radians(6.431585)
thmax = 0.75
phimax = 0.35
chimax = 2.3
eta = np.radians(20.232593)
theta = np.radians(15.416837)
# Bring the CRL to diffracted beam.
crl.goto(theta, eta)

crl.info


------------------------------------------------------------
CRL information in units of [m]
------------------------------------------------------------
Sample to crl distance (d1)     :  0.3782594142025971
CRL to detector distance (d2)   :  6.508990118235542
CRL focal length (f_N)          :  0.3512564714640069
single lens focal length (f)    :  26.825318249049342
Source to detector distance (L) :  6.99764953243814
Lens spacing (T)                :  0.0016
Number of lenses (N)            :  69
Lens radius (R)                 :  5e-05
Refractive Decrement (delta)    :  9.319553925846143e-07
------------------------------------------------------------


In [29]:
beam = GaussianLineBeam(z_std=0.1, energy=energy)  # 100 nm = 0.1 microns

# Beam divergence params
desired_FWHM_N = 0.027 * 1e-3

# Beam wavelength params
sigma_e = (6 * 1e-5) / (2 * np.sqrt(2 * np.log(2)))
epsilon = np.random.normal(0, sigma_e, size=(20000,))
random_energy = energy + epsilon * energy
sigma_lambda = laue.keV_to_angstrom(random_energy).std()
mu_lambda = lambda_0

FWHM_CRL_vertical = 0.556 * 1e-3
FWHM_CRL_horizontal = FWHM_CRL_vertical

# angular_tilt = 0.73 * 1e-3  # perhaps this is what happened in Poulsen 2017?
# the idea is that a slight horizontal titlt of the CRL will cause the
# NA in the horixontal plane to decrease which would explain the rolling curves
# discrepancies.
# dh = (crl.length * np.sin(angular_tilt)) * 1e-6
# FWHM_CRL_horizontal = FWHM_CRL_vertical - 2 * dh

# # TODO: truncation wont help
resolution_function = PentaGauss(
    crl.optical_axis,
    1e-9 / (2 * np.sqrt(2 * np.log(2))),
    # desired_FWHM_N / (2 * np.sqrt(2 * np.log(2))),
    desired_FWHM_N / (2 * np.sqrt(2 * np.log(2))),
    FWHM_CRL_horizontal / (2 * np.sqrt(2 * np.log(2))),
    FWHM_CRL_vertical / (2 * np.sqrt(2 * np.log(2))),
    mu_lambda,
    sigma_lambda,
)
resolution_function.compile()

# Detector size
det_row_count = 272
det_col_count = 272
pixel_size = 0.75

In [30]:
resolution_function._x.cov

array([[6.46936231e-10, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 1.80336880e-19, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.31465586e-10, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.57486218e-08,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        5.57486218e-08]])

In [40]:
np.finfo(float).eps

2.220446049250313e-16

In [35]:
for r in resolution_function._x.cov:
    print((r*1e9).round(4))

[0.6469 0.     0.     0.     0.    ]
[0. 0. 0. 0. 0.]
[0.     0.     0.1315 0.     0.    ]
[ 0.      0.      0.     55.7486  0.    ]
[ 0.      0.      0.      0.     55.7486]
